In [15]:
import numpy as np
import os
import pandas as pd

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker .config import ExportFormat
from tflite_model_maker .text_classifier import AverageWordVecSpec
from tflite_model_maker .text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [16]:
import tensorflow as tf

print(len(tf.config.list_physical_devices('GPU')))

0


In [17]:
df = pd.read_csv('emotion_dataset.csv')

label_map = {0: 'neutral', 1: 'joy', 2: 'sadness', 3: 'angry', 4:'fear'}

df = df.loc[ (df['Emotion'] == 'sadness') | (df['Emotion'] =='fear') | (df['Emotion'] == 'anger')| (df['Emotion'] == 'joy')| (df['Emotion'] == 'neutral')]
print(df['Emotion'].value_counts())
#df=df.drop(df.query("Emotion == 'joy'").sample(frac = 0.5).index)
print(df['Emotion'].value_counts())
df=df.drop(['Text', 'Unnamed: 0'], axis = 1)
df['Clean_Text'] = df['Clean_Text'].str.lower()
df.head()



joy        11045
sadness     6722
fear        5410
anger       4297
neutral     2254
Name: Emotion, dtype: int64
joy        11045
sadness     6722
fear        5410
anger       4297
neutral     2254
Name: Emotion, dtype: int64


,Emotion,Clean_Text
0,neutral,NaN
1,joy,sage act upgrade list tommorow
2,sadness,way homegirl baby funeral man hate funerals sh...
3,joy,eye true hazel eyeand brilliant regular feat...
4,joy,ugh babe hugggzzz u babe naamazed nga ako e...


In [18]:
train = df.sample(frac = 0.8, random_state = 42)
test = df.loc[~df.index.isin(train.index)]

In [19]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [20]:
spec = model_spec.get('mobilebert_classifier')
train_data = DataLoader.from_csv(filename = 'train.csv', text_column = 'Clean_Text', label_column = 'Emotion', model_spec = spec, is_training = True)
test_data = DataLoader.from_csv(filename = 'test.csv', text_column = 'Clean_Text', label_column = 'Emotion', model_spec = spec, is_training = False)


In [21]:
model = text_classifier.create(train_data, model_spec = spec, epochs = 80)

Epoch 1/80
743/743 [==============================] - 2s 2ms/step - loss: 1.5078 - accuracy: 0.3693
Epoch 2/80
743/743 [==============================] - 1s 1ms/step - loss: 1.4900 - accuracy: 0.3694
Epoch 3/80
743/743 [==============================] - 1s 2ms/step - loss: 1.4727 - accuracy: 0.3695
Epoch 4/80
743/743 [==============================] - 1s 2ms/step - loss: 1.4301 - accuracy: 0.3705
Epoch 5/80
743/743 [==============================] - 1s 2ms/step - loss: 1.3712 - accuracy: 0.4126
Epoch 6/80
743/743 [==============================] - 2s 2ms/step - loss: 1.3207 - accuracy: 0.4666
Epoch 7/80
743/743 [==============================] - 1s 2ms/step - loss: 1.2855 - accuracy: 0.4890
Epoch 8/80
743/743 [==============================] - 1s 2ms/step - loss: 1.2584 - accuracy: 0.4974
Epoch 9/80
743/743 [==============================] - 1s 2ms/step - loss: 1.2331 - accuracy: 0.5028
Epoch 10/80
743/743 [==============================] - 1s 2ms/step - loss: 1.2087 - accuracy: 0.5061

In [22]:
loss, acc = model.evaluate(test_data)

186/186 [==============================] - 0s 1ms/step - loss: 0.9330 - accuracy: 0.6867
